## model

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from xgboost import XGBRegressor
import joblib

# ============================
# 1. Load dataset
# ============================
df = pd.read_csv(r"C:\Users\User\Desktop\solar_code\Task\plant1\Plant1_Merged_Final.csv")

# Ensure datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

# Time features
df['hour'] = df['DATE_TIME'].dt.hour
df['minute'] = df['DATE_TIME'].dt.minute
df['time_fraction'] = df['hour'] + df['minute']/60

# Cyclical encoding
df['sin_time'] = np.sin(2 * np.pi * df['time_fraction'] / 24)
df['cos_time'] = np.cos(2 * np.pi * df['time_fraction'] / 24)

In [2]:
df

,DATE_TIME,AC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour,minute,time_fraction,sin_time,cos_time
0,2020-05-15 00:00:00,0.0,25.184316,22.857507,0.0,0,0,0.00,0.000000,1.000000
1,2020-05-15 00:15:00,0.0,25.084589,22.761668,0.0,0,15,0.25,0.065403,0.997859
2,2020-05-15 00:30:00,0.0,24.935753,22.592306,0.0,0,30,0.50,0.130526,0.991445
3,2020-05-15 00:45:00,0.0,24.846130,22.360852,0.0,0,45,0.75,0.195090,0.980785
4,2020-05-15 01:00:00,0.0,24.621525,22.165423,0.0,1,0,1.00,0.258819,0.965926
...,...,...,...,...,...,...,...,...,...,...
3259,2020-06-17 22:45:00,0.0,22.150570,21.480377,0.0,22,45,22.75,-0.321439,0.946930
3260,2020-06-17 23:00:00,0.0,22.129816,21.389024,0.0,23,0,23.00,-0.258819,0.965926
3261,2020-06-17 23:15:00,0.0,22.008275,20.709211,0.0,23,15,23.25,-0.195090,0.980785
3262,2020-06-17 23:30:00,0.0,21.969495,20.734963,0.0,23,30,23.50,-0.130526,0.991445


In [4]:
df.head(30)

,DATE_TIME,AC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour,minute,time_fraction,sin_time,cos_time
0,2020-05-15 00:00:00,0.000000,25.184316,22.857507,0.000000,0,0,0.00,0.000000,1.000000e+00
1,2020-05-15 00:15:00,0.000000,25.084589,22.761668,0.000000,0,15,0.25,0.065403,9.978589e-01
2,2020-05-15 00:30:00,0.000000,24.935753,22.592306,0.000000,0,30,0.50,0.130526,9.914449e-01
3,2020-05-15 00:45:00,0.000000,24.846130,22.360852,0.000000,0,45,0.75,0.195090,9.807853e-01
4,2020-05-15 01:00:00,0.000000,24.621525,22.165423,0.000000,1,0,1.00,0.258819,9.659258e-01
5,2020-05-15 01:15:00,0.000000,24.536092,21.968571,0.000000,1,15,1.25,0.321439,9.469301e-01
6,2020-05-15 01:30:00,0.000000,24.638674,22.352926,0.000000,1,30,1.50,0.382683,9.238795e-01
7,2020-05-15 01:45:00,0.000000,24.873022,23.160919,0.000000,1,45,1.75,0.442289,8.968727e-01
8,2020-05-15 02:00:00,0.000000,24.936930,23.026113,0.000000,2,0,2.00,0.500000,8.660254e-01
9,2020-05-15 02:15:00,0.000000,25.012248,23.343229,0.000000,2,15,2.25,0.555570,8.314696e-01


In [ ]:


# Drop unused columns
df = df.drop(columns=['hour','minute','time_fraction','DATE_TIME'])

# Features and target
X = df.drop(columns=['AC_POWER'])
y = df['AC_POWER'].values   # raw watts

# ============================
# 2. Create sliding windows
# ============================
sequence_length = 6
X_seq, y_seq = [], []

for i in range(len(X) - sequence_length):
    X_seq.append(X.iloc[i:i+sequence_length].values)
    y_seq.append(y[i+sequence_length])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# ============================
# 3. Scale ONLY X
# ============================
scaler_X = MinMaxScaler()
X_flat = X_seq.reshape(-1, X_seq.shape[-1])
X_scaled = scaler_X.fit_transform(X_flat).reshape(X_seq.shape)

# y stays raw
y_raw = y_seq.reshape(-1,1)

In [2]:
y_raw[0:20]

array([[  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [ 90.92916667],
       [593.33869048]])

In [3]:
X

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,sin_time,cos_time
0,25.184316,22.857507,0.0,0.000000,1.000000
1,25.084589,22.761668,0.0,0.065403,0.997859
2,24.935753,22.592306,0.0,0.130526,0.991445
3,24.846130,22.360852,0.0,0.195090,0.980785
4,24.621525,22.165423,0.0,0.258819,0.965926
...,...,...,...,...,...
3259,22.150570,21.480377,0.0,-0.321439,0.946930
3260,22.129816,21.389024,0.0,-0.258819,0.965926
3261,22.008275,20.709211,0.0,-0.195090,0.980785
3262,21.969495,20.734963,0.0,-0.130526,0.991445


In [107]:
X_scaled

array([[[0.32219047, 0.09950559, 0.        , 0.5       , 1.        ],
        [0.31547662, 0.09748388, 0.        , 0.53270156, 0.99892946],
        [0.30545668, 0.09391123, 0.        , 0.5652631 , 0.99572243],
        [0.29942313, 0.0890288 , 0.        , 0.59754516, 0.99039264],
        [0.28430226, 0.08490627, 0.        , 0.62940952, 0.98296291],
        [0.27855073, 0.08075375, 0.        , 0.66071973, 0.97346506]],

       [[0.31547662, 0.09748388, 0.        , 0.53270156, 0.99892946],
        [0.30545668, 0.09391123, 0.        , 0.5652631 , 0.99572243],
        [0.29942313, 0.0890288 , 0.        , 0.59754516, 0.99039264],
        [0.28430226, 0.08490627, 0.        , 0.62940952, 0.98296291],
        [0.27855073, 0.08075375, 0.        , 0.66071973, 0.97346506],
        [0.28545674, 0.08886159, 0.        , 0.69134172, 0.96193977]],

       [[0.30545668, 0.09391123, 0.        , 0.5652631 , 0.99572243],
        [0.29942313, 0.0890288 , 0.        , 0.59754516, 0.99039264],
        [0.28430

In [106]:
y

array([0., 0., 0., ..., 0., 0., 0.], shape=(3264,))

In [5]:
# ============================
# 4. CNN Model
# ============================
timesteps = X_scaled.shape[1]
n_features = X_scaled.shape[2]

inputs = Input(shape=(timesteps, n_features))
x = layers.Conv1D(64, 3, activation='relu', padding="causal")(inputs)
x = layers.Conv1D(32, 3, activation='relu', padding="causal")(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)

cnn_model = Model(inputs, outputs)
cnn_model.compile(optimizer='adam', loss='mse')

In [6]:
cnn_model

<Functional name=functional_1, built=True>

In [8]:
Model

keras.src.models.model.Model

In [9]:
cnn_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 6, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 6, 64)          │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 6, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,377 (36.63 KB)

 Trainable params: 9,377 (36.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:


print("\nTraining CNN...")
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)
history = cnn_model.fit(
    X_scaled, y_raw,                 
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)
# ============================
# 5. Extract CNN features
# ============================
cnn_feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
cnn_features = cnn_feature_extractor.predict(X_scaled)
print("Extracted CNN feature shape:", cnn_features.shape)


Training CNN...
Epoch 1/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 114415240.0000 - val_loss: 71392168.0000
Epoch 2/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 41622152.0000 - val_loss: 17709674.0000
Epoch 3/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15497203.0000 - val_loss: 10023668.0000
Epoch 4/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 10923913.0000 - val_loss: 7835457.0000
Epoch 5/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8724520.0000 - val_loss: 6923797.0000
Epoch 6/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7630466.0000 - val_loss: 6465373.0000
Epoch 7/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7178088.5000 - val_loss: 6500450.0000
Epoch 8/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7103515.5000 - val_loss: 6302011.0000
Epoch 9/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7007363.0000 - val_loss: 6285144.5000
Epoch 10/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6985835.0000 - val_los

In [108]:
cnn_features 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(3258, 64), dtype=float32)

In [96]:
# ============================
# 6. Train/Test split on Plant1
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    cnn_features, y_raw, test_size=0.2, random_state=42, shuffle=True
)

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42
)


print("\nTraining XGBoost on Plant1 (80% train, 20% test)...")
xgb_model.fit(X_train, y_train)

# ============================
# 7. Evaluate
# ============================
y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"📊 MSE: {mse:.2f}")
print(f"📊 RMSE: {rmse:.2f}")
print(f"📊 R²: {r2:.4f}")
print(f"📊 MAE: {mae:.2f}")



Training XGBoost on Plant1 (80% train, 20% test)...
📊 MSE: 6686872.57
📊 RMSE: 2585.90
📊 R²: 0.9063
📊 MAE: 1376.32


In [98]:
# ============================
# 8. Save models
# ============================
joblib.dump(scaler_X, r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\scaler_X.pkl")
cnn_model.save(r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\cnn_model.h5", include_optimizer=False)
xgb_model.save_model(r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\xgb_model.json")
print("✅ Models and scaler saved successfully!")


✅ Models and scaler saved successfully!


## inference

In [1]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model, Model
from xgboost import XGBRegressor

# ============================
# 1. Load saved objects
# ============================
scaler_X = joblib.load(r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\scaler_X.pkl")
cnn_model = load_model(r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\cnn_model.h5", compile=False)

# CNN feature extractor (same as training)
cnn_feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

xgb_model = XGBRegressor()
xgb_model.load_model(r"C:\Users\User\Desktop\solar_code\Task\plant1\mod\xgb_model.json")


# ============================
# 2. Preprocessing function
# ============================
def preprocess_input(df_input, sequence_length=6):
    df_input['DATE_TIME'] = pd.to_datetime(df_input['DATE_TIME'])

    # Time features
    df_input['hour'] = df_input['DATE_TIME'].dt.hour
    df_input['minute'] = df_input['DATE_TIME'].dt.minute
    df_input['time_fraction'] = df_input['hour'] + df_input['minute'] / 60.0

    # Cyclical encoding
    df_input['sin_time'] = np.sin(2 * np.pi * df_input['time_fraction'] / 24)
    df_input['cos_time'] = np.cos(2 * np.pi * df_input['time_fraction'] / 24)

    df_input = df_input.drop(columns=['hour', 'minute', 'time_fraction', 'DATE_TIME'])

    # Build sliding window
    X_seq = []
    for i in range(len(df_input) - sequence_length + 1):   # ✅ Fix here
        X_seq.append(df_input.iloc[i:i+sequence_length].values)

    X_seq = np.array(X_seq)

    # Scale
    X_flat = X_seq.reshape(-1, X_seq.shape[-1])
    X_scaled = scaler_X.transform(X_flat).reshape(X_seq.shape)

    return X_scaled



# ============================
# 3. Inference function
# ============================
def predict_ac_power(new_data):
    """
    new_data: DataFrame with columns [DATE_TIME, AMBIENT_TEMPERATURE, MODULE_TEMPERATURE, IRRADIATION]
    returns: predicted AC_POWER
    """
    X_scaled = preprocess_input(new_data)

    # CNN → features
    cnn_features = cnn_feature_extractor.predict(X_scaled)

    # XGB → prediction
    y_pred = xgb_model.predict(cnn_features)

    return y_pred


# ============================
# 4. Example Usage
# ============================
# Sample: provide at least 6 consecutive rows (for sliding window)

sample_data = pd.DataFrame({
    "DATE_TIME": [
        "2020-05-15 06:00:00",
        "2020-05-15 06:15:00",
        "2020-05-15 06:30:00",
        "2020-05-15 06:45:00",
        "2020-05-15 07:00:00",
        "2020-05-15 07:15:00",
    ],
    "AMBIENT_TEMPERATURE": [24.08, 24.01, 23.97, 24.21, 24.53, 24.82],
    "MODULE_TEMPERATURE": [22.20, 22.35, 22.89, 24.44, 27.19, 28.88],
    "IRRADIATION": [0.0058, 0.0223, 0.0494, 0.0954, 0.1419, 0.1547]
})

preds = predict_ac_power(sample_data)
print("🔮 Predicted AC Power:", preds)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
🔮 Predicted AC Power: [5463.6094]
